In [1]:
# !!!!!!!!! DO NOT RUN IN MINERL CONDA ENVIRONMENT - this environment is very fragile and minerl is easily borken with chagned dependenceies but this code will want to update its dependencies and break it!
if False:
    !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
    !pip install opencv-python 
    !pip install git+https://github.com/openai/CLIP.git
    !pip install scikit-learn

In [3]:
import clip
from PIL import Image
import torch as th
import cv2
import random
from sklearn import svm as sklearn_svm
import pickle

C:\Users\idmi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
th.seed=0
random.seed(0)

In [6]:
# LOAD CLIP
device = "cuda" if th.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50x64", device=device)
model.to(device)

CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)


In [10]:
# LOAD DATASET
with open('spamham_samples/frames_labels.txt') as file:
    frames_labels = file.readlines()
    random.shuffle(frames_labels)
num_samples = len(frames_labels)

# load labels
y_labels = []
for i in range(num_samples):
    label = frames_labels[i].split(',')[1]
    y_labels.append(label)

# load images
# from images, create image features from CLIP
x_image_features = th.zeros([num_samples, 1024]).to(device)
for i in range(num_samples):
    image_name = 'spamham_samples/frames/' + frames_labels[i].split(',')[0] + '.jpeg'
    with Image.open(image_name) as image:
        image = preprocess(image).unsqueeze(0).to(device)
    with th.no_grad():
        x_image_features[i,:] = model.encode_image(image)

    if i%(num_samples//100)==0:
        print('=',end='')
print()



In [51]:
# turn to CPU train/test features for SVM
split=0.9
x_train = x_image_features[int(num_samples*split):].cpu()
y_train = y_labels[int(num_samples*split):]
x_test = x_image_features[:int(num_samples*split)].cpu()
y_test = y_labels[:int(num_samples*split)]

In [11]:
# save image feature dataset for faster use later just in case
#th.save(x_image_features, 'x_image_features')

In [55]:
# for final training, train on whole dataset
#x_train = x_image_features.cpu()
#y_train = y_labels

In [56]:
### INITIALIZE SVM
svm = sklearn_svm.SVC(kernel='rbf', gamma='scale', C=8)

In [58]:
# FIT SVM TO IMAGE FEATURES -> LABELS
svm.fit(x_train, y_train)
svm.score(x_test, y_test)

0.9008035831906205

In [59]:
pickle.dump(svm, open('SVM', 'wb'))